# Sparkify Project Pipeline - Full Dataset on GCP Cluster

This Jupyter Notebook contains the execution script for loading the Sparkify dataset (12GB), extracting features, running machine learning models to predict 'churn' and finally tuning the model to achieve best results.

This Notebook is run on larger dataset to test and present various concepts on a Spark cluster. Some of the exploration tasks have been trimmed from this Notebook to give a better performance with large dataset.

## Import Dependencies & Libraries 

In [1]:
# Starter code
from pyspark.sql import SparkSession

In [2]:
# Create spark session
spark = SparkSession \
        .builder \
        .appName("Sparkify") \
        .getOrCreate()

In [3]:
# import libraries
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, PCA
from pyspark.ml.feature import Normalizer, MinMaxScaler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## Feature Extraction

Following are all the functions that are used to extract features from the dataset and make a dataframe ready for machine learning algorithms.

In [4]:
def load_data(filename):
    
    '''
    Funtion to load data and remove null, empty strings.
    INPUT
    filename = name of file as well the path
    OUTPUT
    df - a spark dataframe with no null rows for primary key
    '''
    
    df = spark.read.json(filename)
    
    # remove empty string from userId
    df = df.filter(df.userId != "")
    
    print("Count of rows in dataframe: {}".format(df.count()))
          
    return df

In [5]:
def add_date_columns(df):
    '''
    Funtion to add date/time related columns to the dataframe.
    INPUT - a spark dataframe
    OUPUT - a spark dataframe with calculated fields: hour, month, year, day
    '''
    # create a function to get hour, month, year, day from timestamp
    get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
    get_month = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).month)
    get_year = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).year)
    get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).day)

    
    
    # add hour, month, year, day, date columns to the dataframe
    df = df.withColumn("hour", get_hour(df.ts))
    df = df.withColumn("month", get_month(df.ts))
    df = df.withColumn("year", get_year(df.ts))
    df = df.withColumn("day", get_day(df.ts))
    df = df.withColumn("date", from_unixtime(df.ts/1000).cast(DateType()))
    
    
    return df

In [6]:
def get_flag_churn(df):
    '''
    Funtion to add churn flag that identifies whether a user has churned.
    INPUT - a spark dataframe
    OUPUT - a spark dataframe with userId of each user, churn flag and gender
    '''
    
    # function to flag 'cancellation confirmation' event
    flag_cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
    df = df.withColumn("churn", flag_cancel_event("page"))
    user_churn = df.groupBy("userId").agg({"churn":"max", "gender":"max"})\
                           .withColumnRenamed("max(churn)", "label")\
                           .withColumnRenamed("max(gender)", "gender")
    
    #print("Count of rows: {}".format(user_churn.count()))
    return user_churn

In [7]:
def get_latest_level(df):
    '''
    Funtion to find the latest level of each user.
    INPUT - a spark dataframe
    OUPUT - a spark dataframe with userId of each user and latest level
    '''
    # use timestamp to identify the most latest record for a user
    # sort timestamp in descending order and than drop duplicates to get the last row for each user
    latest_level = df.select(["userId", "level", "ts"])\
                        .orderBy(desc("ts"))\
                        .dropDuplicates(["userId"])\
                        .select(["userId", "level"])
    #print("Count of rows: {}".format(latest_level.count()))
    return latest_level

In [8]:
def get_states(df):
    '''
    Funtion to clean location column and return cleaned state names.
    INPUT - a spark dataframe
    OUPUT - a spark dataframe with userId of each user and state from location
    '''
    # get location of each user
    state_data = df.groupBy("userId").agg({"location":"max"}).withColumnRenamed("max(location)", "state")
    # extract state
    state_data = state_data.withColumn("state", split(col("state"),',').getItem(1))
    
    #print("Count of rows: {}".format(state_data.count()))
    
    return state_data

In [9]:
def get_device(df):
    '''
    Funtion to clean userAgent column and return cleaned device/os names.
    INPUT:
    df - a spark dataframe
    OUPUT:
    device_data - a spark dataframe with userId of each user and device/os name
    '''
      
    device_data = df.groupBy("userId").agg({"userAgent":"max"}).withColumnRenamed("max(userAgent)", "device")
    device_data = device_data.withColumn("device", regexp_extract(col("device"), r'\((.*?)\)', 1));
    
    device_data = device_data.withColumn("device", split(col("device"),';').getItem(0))
    device_data = device_data.withColumn("device", split(col("device"),'NT').getItem(0))
    
    device_data = device_data.withColumn("device", trim(device_data.device))
    
    #print("Count of rows: {}".format(device_data.count()))
    
    return device_data

In [10]:
def get_days_since_reg(df):
    '''
    Funtion to get the number of days since registration
    INPUT:
    df - a spark dataframe
    OUPUT:
    days_since_reg - a spark dataframe with userId of each user and days since registration
    '''
    
    # replace null in 'registration' or 'ts' field with 0
    df = df.fillna(0, subset=['registration', 'ts'])
    
    get_subtract_ts = udf(lambda x, y: datetime.datetime.fromtimestamp((y - x) / 1000.0).day, IntegerType())

    days_since_reg = df.groupBy("userId").agg({"registration":"max", "ts":"max"})\
                                         .withColumnRenamed("max(registration)", "max_reg")\
                                         .withColumnRenamed("max(ts)", "max_ts")
    
    days_since_reg = days_since_reg.withColumn("days_since_reg", when(col('max_reg') != 0, get_subtract_ts(col('max_reg'), col('max_ts'))).otherwise(0))
    
    days_since_reg = days_since_reg.drop("max_reg")
    days_since_reg = days_since_reg.drop("max_ts")
    
    #print("Count of rows: {}".format(days_since_reg.count()))
    
    return days_since_reg

In [11]:
def get_avg_count_session(df):
    '''
    Funtion to get the average number of sessions per month for a user
    INPUT:
    df - a spark dataframe
    OUPUT:
    avg_count_session - a spark dataframe with userId of each user and average monthly count of sessions
    '''
    mon_count_session = df.groupBy("userId", "month").agg(countDistinct("sessionId"))\
                          .groupBy("userId").agg(avg("count(DISTINCT sessionId)"))\
                          .withColumnRenamed("avg(count(DISTINCT sessionId))", "avg_mon_session_count")
    
    #print("Count of rows: {}".format(mon_count_session.count()))
    
    return mon_count_session

In [12]:
def get_avg_duration_session(df):
    '''
    Funtion to get the average number of sessions per month for a user
    INPUT:
    df - a spark dataframe
    OUPUT:
    avg_duration_session - a spark dataframe with userId of each user and average monthly count of sessions
    '''
    
    mon_sess_duration = df.groupBy("userId", "month").agg(max("ts"), min("ts"))\
                          .withColumn("duration", (col("max(ts)") - col("min(ts)"))/1000)\
                          .groupBy("userId").agg(avg("duration"))\
                          .withColumnRenamed("avg(duration)", "avg_mon_sess_duration")
    
    #print("Count of rows: {}".format(mon_sess_duration.count()))
    
    return mon_sess_duration

In [13]:
def get_avg_page_views(df):
    '''
    Funtion to get the average number of page views per date and month for a user
    INPUT:
    df - a spark dataframe
    OUPUT:
    avg_page_views - a spark dataframe with userId of each user and average monthly page views with each page pivoted
    '''
    # --------------- Monthly views -----------------:
    # get avg of page views per month
    avg_mon_page_views = df.groupBy("userId", "page", "month").agg(count("page"))\
                           .groupBy("userId", "page").agg(avg("count(page)"))\
                           .withColumnRenamed("avg(count(page))", "avg_page_count")
    
    # clean up page column
    avg_mon_page_views = avg_mon_page_views.withColumn("page", trim(avg_mon_page_views.page))
    avg_mon_page_views = avg_mon_page_views.withColumn("page", regexp_replace(avg_mon_page_views.page, " ", "_"))
    
    # add prefix to each page name
    add_prefix = udf(lambda x: "avg_mon_" + x)
    avg_mon_page_views = avg_mon_page_views.withColumn("page", add_prefix(avg_mon_page_views.page))
    
    # pivot page names to get columns
    avg_mon_page_views = avg_mon_page_views.groupBy("userId").pivot("page").max("avg_page_count")
    avg_mon_page_views = avg_mon_page_views.fillna(0)
    
    # ---------------- Daily views --------------------:
    # get avg of page views per day
    avg_daily_page_views = df.groupBy("userId", "page", "date").agg(count("page"))\
                           .groupBy("userId", "page").agg(avg("count(page)"))\
                           .withColumnRenamed("avg(count(page))", "avg_daily_page_count")
    
    # clean up page column
    avg_daily_page_views = avg_daily_page_views.withColumn("page", trim(avg_daily_page_views.page))
    avg_daily_page_views = avg_daily_page_views.withColumn("page", regexp_replace(avg_daily_page_views.page, " ", "_"))
    
    # add prefix to each page name
    add_prefix_daily = udf(lambda x: "avg_daily_" + x)
    avg_daily_page_views = avg_daily_page_views.withColumn("page", add_prefix_daily(avg_daily_page_views.page))
    
    # pivot page names to get columns
    avg_daily_page_views = avg_daily_page_views.groupBy("userId").pivot("page").max("avg_daily_page_count")
    avg_daily_page_views = avg_daily_page_views.fillna(0)

    # join daily and monthly views.
    avg_page_views = avg_daily_page_views.join(avg_mon_page_views, on="userId")

    #print("Count of rows: {}".format(avg_mon_page_views.count()))

    
    return avg_page_views

In [14]:
def extract_features(df):
    '''
    Funtion to extract feautres from the log data.
    INPUT:
    df - a spark dataframe
    OUPUT:
    df - a spark dataframe with userId as set of unique rows and columns with features extracted
    '''

    # add date columns
    df = add_date_columns(df)

    # get churn and gender columns
    user_label = get_flag_churn(df)

    # get latest level of the user
    latest_level = get_latest_level(df)

    # extract states from location
    states = get_states(df)

    # extract device/os from userAgent
    device = get_device(df)

    # get days since registration
    days_since_reg = get_days_since_reg(df)

    # get average monthly count of sessions per user
    avg_count_session = get_avg_count_session(df)

    # get average monthly duration of sessions per user
    avg_duration_session = get_avg_duration_session(df)

    # get average monthly page views per page per user
    avg_page_views = get_avg_page_views(df)


    # join all extracted tables
    user_data_all = user_label.join(latest_level, on="userId")\
                              .join(states, on="userId")\
                              .join(device, on="userId")\
                              .join(days_since_reg, on="userId")\
                              .join(avg_count_session, on="userId")\
                              .join(avg_duration_session, on="userId")\
                              .join(avg_page_views, on="userId")

    # check count
    x1 = user_data_all.count()
    x2 = event_data.select("userId").dropDuplicates().count()
    if (x1 == x2):
      print("The number of rows in final merged table ({}) matches number of unique users ({}).".format(x1, x2))
    else:
      print("The numebr of rows in final merged tables ({}) do not match number of unique users ({}).".format(x1, x2))


    # check for nans and nulls
    user_data_all.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in user_data_all.columns]).show()

    return user_data_all

## Modeling

### Data transformation for ML algorithms

In [15]:
def transform_data(df, numerical_col, categorical_col, pca=10):
    '''
    Function to transform categorical fields and create features vector. The function uses Pipeline to convert string to numerical, 
    vectorize, standardize and perform PCA.
    INPUT:
    df - a spark dataframe
    numerical_col - a list of numerical columns that should be used in ML models
    categorical_col - a list of categorical columns that should be used in ML models
    pca - a number of principal components to retain for ML models. Default value is 10.
    OUPUT:
    df - a spark dataframe with transformed data containing features, scaledFeatures and pcaFeatures vector
    '''
    
    # Define stringIndexer for converting categorical to numerical variables
    # Apply VectorAssembler to create feature vector
    # Normalize data before performing PCA
    # Perform PCA for feature selection
    # Use pipeline to create ML ready data
    

    # convert categorical columns to numerical
    stages = []
    for col in categorical_col:
        stringIndexer = StringIndexer(inputCol = col, outputCol = col + '_index', handleInvalid = 'keep')
        #encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[col + "_classVec"])
        #stages += [stringIndexer, encoder]
        stages += [stringIndexer]

    # vectorize columns
    assemblerInputs = [c + "_index" for c in categorical_col] + numerical_col
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]
    

    # Rescale feature vector
    # scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",   withStd=True, withMean=False)
    scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
    stages += [scaler]
    
    pca = PCA(k=pca, inputCol="scaledFeatures", outputCol="pcaFeatures")
    stages += [pca]


    # use pipeline to transform data
    cols = df.columns
    pipeline = Pipeline(stages = stages)
    pipelineModel = pipeline.fit(df)
    df = pipelineModel.transform(df)
    
    #select new 'feature' column and rest of columns
    selectedCols = ['features', 'scaledFeatures', 'pcaFeatures'] + cols
    df = df.select(selectedCols)

    return df

### Baseline models testing

In [16]:
def model_testing(model, train, test):
    '''
    Funtion to test baseline machine learning algorithms
    INPUT:
    model - instantiated model object
    train - training dataset
    test - testing dataset
    OUPUT:
    score - F1 score to measure performance of the algorithm
    '''
       
    cl_model = model.fit(train)
    predict_train = cl_model.transform(train)
    predict_test = cl_model.transform(test)

    # Because of imbabalnced dataset, it is preferred to use F1 score as evaluation metric
    evaluator = MulticlassClassificationEvaluator(metricName='f1')
    score_train = evaluator.evaluate(predict_train)
    score_test = evaluator.evaluate(predict_test)

    return score_train, score_test

### Parameter tuning

In [11]:
def model_tuning(train, test, model, paramGrid, numFolds):
    '''
    Funtion to tune a machine learning algorithm using cross validation
    INPUT:
    model - instantiated model object
    paramGrid - hyper parameters associated with the model used for tuning
    numFolds - number of cross validation folds
    train - training dataset
    test - testing dataset
    
    OUPUT:
    score_train - F1 score to measure performance of the algorithm on training dataset
    score_test - F1 score to measure performance of the algorithm on testing dataset
    predict_train - dataframe with prediction results from transformation on training dataset
    predict_test - dataframe with prediction results from transformation on testing dataset
    bestModel - best model object to get the best parameters from the paramGrid
    '''
    
    # instantiate evaluator with F1 score as metric
    evaluator = MulticlassClassificationEvaluator(metricName="f1")

    # instantiate cross validation object
    cv = CrossValidator(estimator=model, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=numFolds)
    
    # fit and transform
    cvmodel = cv.fit(train)
    predict_train = cvmodel.transform(train)
    predict_test = cvmodel.transform(test)
    
    # get the f1 score
    score_train = evaluator.evaluate(predict_train)
    score_test = evaluator.evaluate(predict_test)
    
    bestModel = cvmodel.bestModel
    
    return score_train, score_test, predict_train, predict_test, bestModel
    

## Execution

### 1. Load dataset

In [18]:
# link to mini dataset
path = "gs://sparkify-bucket/data/sparkify_event_data.json"

# load Spark dataframe
event_data = load_data(path)

Count of rows in dataframe: 26259199


In [19]:
# check for nans and nulls
event_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in event_data.columns]).show()

+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
| artist|auth|firstName|gender|itemInSession|lastName| length|level|location|method|page|registration|sessionId|   song|status| ts|userAgent|userId|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
|5408927|   0|   778479|778479|            0|  778479|5408927|    0|  778479|     0|   0|      778479|        0|5408927|     0|  0|   778479|     0|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+



In [20]:
event_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



### 2. Extract features

In [ ]:
%%time
user_data_all = extract_features(event_data)

The number of rows in final merged table (22278) matches number of unique users (22278).
+------+------+-----+-----+-----+------+--------------+---------------------+---------------------+---------------+--------------------+-------------------------+----------------+-----------------------------------+-------------------+---------------+--------------+--------------+---------------+----------------+------------------+------------------+---------------------+-----------------------+------------------+--------------------------+-----------------------------+------------------------+---------------------+-------------------+-----------------+-------------+------------------+-----------------------+--------------+---------------------------------+-----------------+-------------+------------+------------+-------------+--------------+----------------+----------------+-------------------+---------------------+----------------+------------------------+---------------------------+-------------

In [ ]:
# check schema for all columns
user_data_all.printSchema()

root
 |-- userId: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- label: integer (nullable = true)
 |-- level: string (nullable = true)
 |-- state: string (nullable = true)
 |-- device: string (nullable = true)
 |-- days_since_reg: integer (nullable = true)
 |-- avg_mon_session_count: double (nullable = true)
 |-- avg_mon_sess_duration: double (nullable = true)
 |-- avg_daily_About: double (nullable = false)
 |-- avg_daily_Add_Friend: double (nullable = false)
 |-- avg_daily_Add_to_Playlist: double (nullable = false)
 |-- avg_daily_Cancel: double (nullable = false)
 |-- avg_daily_Cancellation_Confirmation: double (nullable = false)
 |-- avg_daily_Downgrade: double (nullable = false)
 |-- avg_daily_Error: double (nullable = false)
 |-- avg_daily_Help: double (nullable = false)
 |-- avg_daily_Home: double (nullable = false)
 |-- avg_daily_Login: double (nullable = false)
 |-- avg_daily_Logout: double (nullable = false)
 |-- avg_daily_NextSong: double (nullable = false

### 3. Data Prep before ML

In [23]:
# get list of categorical columns
str_cols = [item[0] for item in user_data_all.dtypes if item[1].startswith('string')]
# remove userId since it's not required for modeling
str_cols.remove('userId')

# get list of numerical columns
num_cols = [item[0] for item in user_data_all.dtypes if not item[1].startswith('string')]
# remove label column since it's what we are predicting
num_cols.remove('label')

Remove 'Cancel' and 'Cancellation_Confirmation' pages since we want to predict churn before we reach these pages.

In [25]:
# remove 'Cancel' and 'Cancellation_Confirmation' pages since we want to predict churn before we reach these pages.
num_cols.remove('avg_daily_Cancel')
num_cols.remove('avg_daily_Cancellation_Confirmation')

num_cols.remove('avg_mon_Cancel')
num_cols.remove('avg_mon_Cancellation_Confirmation')

#### 3.3. Transform data using Pipelines

Transform dataset to get scaledFeatures and default PCA=10. PCA analysis will be performed later on 'scaledFeatures' to ascertain the best value for PCA features.

In [ ]:
%%time
user_data_all_trans = transform_data(user_data_all, num_cols, str_cols, 10)

CPU times: user 1.36 s, sys: 565 ms, total: 1.92 s
Wall time: 42min 14s


#### 3.2 Imbalanced dataset

A dataset is considered imbalanced if the ratio of one class is much higher than another class. In this dataset, the number of users who have churned are way less than number of active users. This poses a problem with classification models where all classes are not represented equally.

In [ ]:
major_df = user_data_all_trans.filter(user_data_all_trans.label == 0)
minor_df = user_data_all_trans.filter(user_data_all_trans.label == 1)
ratio_df = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio_df))

ratio: 3


The data is heavily skewed towards label == 0 i.e. there are way more active users than churned users. Because the prediction classs is imbalanced, the ML models will be heavily skewed towards predicting label 0. In order to circumvent this situation, 'weights' column is created to provide more weight to the minority class. 

In [ ]:
# add new column weights
# use user defined ratio field to provide appropriate weights to majority and minority class
ratio = 1/ratio_df
user_data_all_trans = user_data_all_trans.withColumn("weights", when(user_data_all_trans.label == 1, ratio).otherwise(1-ratio))

In [ ]:
user_data_all_trans.head()

Row(features=DenseVector([0.0, 1.0, 4.0, 0.0, 19.0, 11.0, 1603889.5, 0.0, 1.75, 2.2727, 3.0, 1.0, 1.6, 2.4444, 0.0, 1.3636, 51.1, 0.0, 3.7, 1.0, 1.8, 1.0, 0.0, 1.0, 2.0625, 3.5333, 1.8, 0.0, 7.0, 12.5, 3.0, 1.5, 4.0, 22.0, 0.0, 7.5, 511.0, 0.0, 37.0, 1.0, 4.5, 1.0, 0.0, 1.0, 16.5, 26.5, 4.5]), scaledFeatures=DenseVector([0.0, 1.0, 0.04, 0.0, 0.6129, 0.0001, 0.5989, 0.0, 0.1346, 0.1748, 0.3333, 0.0667, 0.0038, 0.0004, 0.0, 0.1948, 0.1703, 0.0, 0.1, 0.2, 0.2571, 0.5, 0.0, 0.5, 0.2292, 0.1823, 0.2571, 0.0, 0.0511, 0.0698, 0.0451, 0.0033, 0.0003, 0.0001, 0.0, 0.0893, 0.0855, 0.0, 0.1462, 0.0769, 0.1125, 0.1667, 0.0, 0.1667, 0.2143, 0.0472, 0.2368]), pcaFeatures=DenseVector([-0.1748, 1.4498, 0.6747, 0.0235, -0.2046, 0.0358, -0.0505, -0.0181, 0.2288, -0.4635]), userId=u'1000280', gender=u'M', label=1, level=u'free', state=u' OH', device=u'Windows', days_since_reg=19, avg_mon_session_count=11.0, avg_mon_sess_duration=1603889.5, avg_daily_About=0.0, avg_daily_Add_Friend=1.75, avg_daily_Add_to_

'weights' column will be used in logistic regression becuase the model itself has an in-build parameter to handle imbalanced dataset.

### 5. Split data into training and testing datasets

In [ ]:
%%time
# split data for training and testing datasets
train, test = user_data_all_trans.randomSplit([0.7, 0.3], seed=42)
print(train.count())
print(test.count())

15530
6748
CPU times: user 522 ms, sys: 267 ms, total: 790 ms
Wall time: 17min 3s


#### 5.1 Save data before running ML models for later load.

In [ ]:
# # save parquet file - save data
train.write.parquet("gs://sparkify-bucket/data/train.parquet")

In [ ]:
# # save parquet file - save data
test.write.parquet("gs://sparkify-bucket/data/test.parquet")

In [4]:
# using SQLContext to read parquet file
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# to read parquet file
train = sqlContext.read.parquet("gs://sparkify-bucket/data/train.parquet")
test = sqlContext.read.parquet("gs://sparkify-bucket/data/test.parquet")

### 6. Run baseline models

#### Instantiate model objects
Run models with default parameters.

In [8]:
# Logistic Regression
lr_model = LogisticRegression(featuresCol = 'pcaFeatures', labelCol = 'label')
lr_model_w = LogisticRegression(featuresCol = 'pcaFeatures', labelCol = 'label', weightCol='weights')

# Gradient Boosting Trees (GBT)
gbt_model = GBTClassifier(featuresCol = 'pcaFeatures', labelCol = 'label')

# Decision Tree Classifier
dt_model = DecisionTreeClassifier(featuresCol = 'pcaFeatures', labelCol = 'label')

# RandomForest Classifier
rf_model = RandomForestClassifier(featuresCol = 'pcaFeatures', labelCol = 'label')

#### F1 score for each model

f1 score is a measure of accuracy that uses both precision and recall. 

f1 = 2*((precision*recall)/(precision+recall))

Since we are dealing with imbalanced dataset where churned users much less than active users, it's best to use f1 score rather than accuracy which may provide high accuracy by just predicting majority class.

In [ ]:
%%time
# Logistic Regression
lr_score_train, lr_score_test = model_testing(lr_model, train, test)
print("f1 score for Logistic Regression: train dataset {}, test dataset {}".format(lr_score_train, lr_score_test))

f1 score for Logistic Regression: train dataset 0.710391110612, test dataset 0.714728010953
CPU times: user 1.14 s, sys: 530 ms, total: 1.67 s
Wall time: 27min 9s


In [ ]:
%%time
# Logistic Regression with weights
lr_score_train_w, lr_score_test_w = model_testing(lr_model_w, train, test)
print("f1 score for Logistic Regression: train dataset {}, test dataset {}".format(lr_score_train_w, lr_score_test_w))

f1 score for Logistic Regression: train dataset 0.673677046346, test dataset 0.677823686155
CPU times: user 899 ms, sys: 387 ms, total: 1.29 s
Wall time: 26min 28s


We observe not a lot of diference between Logistic Regression using 'weights' and Logistic Regression without 'weights'.

In [ ]:
%%time
gbt_score_train, gbt_score_test = model_testing(gbt_model, train, test)
print("f1 score for GBT Classifier: train dataset {}, test dataset {}".format(gbt_score_train, gbt_score_test))

f1 score for GBT Classifier: train dataset 0.773517400113, test dataset 0.769259875381
CPU times: user 2.81 s, sys: 1.42 s, total: 4.23 s
Wall time: 27min 2s


In [ ]:
%%time
dt_score_train, dt_score_test = model_testing(dt_model, train, test)
print("f1 score for Decision Tree Classifier: train dataset {}, test dataset {}".format(dt_score_train, dt_score_test))

f1 score for Decision Tree Classifier: train dataset 0.740949527496, test dataset 0.743873990648
CPU times: user 1.33 s, sys: 531 ms, total: 1.86 s
Wall time: 33min 32s


In [ ]:
%%time
rf_score_train, rf_score_test = model_testing(rf_model, train, test)
print("f1 score for Random Forest Classifier: train dataset {}, test dataset {}".format(rf_score_train, rf_score_test))

f1 score for Random Forest Classifier: train dataset 0.724383305188, test dataset 0.725493733818
CPU times: user 1.19 s, sys: 552 ms, total: 1.74 s
Wall time: 33min 40s


Performance of classification models increased drastically as we ran these models on larger dataset.
In terms of performance, with just base settings, the GBT classifier outperformed the rest of the classifiers.

Now we can perform hyper parameter tuning to see if we can further improve the models.

### 7. Parameter tuning

#### Paramter tuning for Logistic Regression:

In [ ]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr_model.aggregationDepth,[2,5])\
    .addGrid(lr_model.elasticNetParam,[0.0, 0.5])\
    .addGrid(lr_model.maxIter,[10, 100])\
    .addGrid(lr_model.regParam,[0.01, 0.1]) \
    .build()

In [ ]:
%%time
score_train_lr, score_test_lr, predict_train_lr, predict_test_lr, bestModel_lr = model_tuning(train, test, lr_model, paramGrid, 5)

CPU times: user 53.5 s, sys: 29 s, total: 1min 22s
Wall time: 2h 25min 8s


In [ ]:
print("f1 train score: {}".format(score_train_lr))
print("f1 test score: {}".format(score_test_lr))
print("Best parameter for Aggregation depth: {}".format(bestModel_lr._java_obj.getAggregationDepth()))
print("Best parameter for Elastic Net param: {}".format(bestModel_lr._java_obj.getElasticNetParam()))
print("Best parameter for iterations: {}".format(bestModel_lr._java_obj.getMaxIter()))
print("Best parameter for param: {}".format(bestModel_lr._java_obj.getRegParam()))

f1 train score: 0.711798861763
f1 test score: 0.717922004117
Best parameter for Aggregation depth: 2
Best parameter for Elastic Net param: 0.0
Best parameter for iterations: 10
Best parameter for param: 0.01


In [ ]:
# check whether prediction consists of both labels.
predict_test_lr.select("prediction").dropDuplicates().show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



In [ ]:
predict_test_lr.select("userId", "label", "prediction").show()

+-------+-----+----------+
| userId|label|prediction|
+-------+-----+----------+
|1506897|    0|       0.0|
|1553683|    0|       0.0|
|1617595|    1|       0.0|
|1804292|    0|       0.0|
|1532306|    0|       0.0|
|1744249|    0|       0.0|
|1351489|    0|       0.0|
|1190352|    0|       0.0|
|1828442|    0|       0.0|
|1059049|    0|       0.0|
|1358765|    1|       0.0|
|1200956|    0|       0.0|
|1500901|    0|       0.0|
|1880560|    0|       0.0|
|1180406|    0|       0.0|
|1612069|    0|       0.0|
|1142513|    0|       0.0|
|1699838|    0|       0.0|
|1917123|    0|       0.0|
|1927371|    0|       0.0|
+-------+-----+----------+
only showing top 20 rows



We don't see any improvedment in Logistic regression test score.

#### Parameter tuning for Random forest classifier:

In [ ]:
# param grid for random forest classifier
rf_paramGrid = ParamGridBuilder()\
              .addGrid(rf_model.maxDepth,[5, 10, 15])\
              .addGrid(rf_model.numTrees,[50, 100, 200])\
              .addGrid(rf_model.maxBins,[50, 100])\
              .build()

In [ ]:
%%time
score_train_rf, score_test_rf, predict_train_rf, predict_test_rf, bestModel_rf = model_tuning(train, test, rf_model, rf_paramGrid, 5)

CPU times: user 1min 8s, sys: 24.6 s, total: 1min 33s
Wall time: 7h 11min 19s


In [ ]:
print("f1 train score: {}".format(score_train_rf))
print("f1 test score: {}".format(score_test_rf))
print("Best parameter for max depth: {}".format(bestModel_rf._java_obj.getMaxDepth()))
print("Best parameter for number of tress: {}".format(bestModel_rf._java_obj.getNumTrees()))
print("Best parameter for number of bins: {}".format(bestModel_rf._java_obj.getMaxBins()))

f1 train score: 0.912099551257
f1 test score: 0.83646658194
Best parameter for max depth: 15
Best parameter for number of tress: 200
Best parameter for number of bins: 50


In [ ]:
# check whether prediction consists of both labels.
predict_test_rf.select("prediction").dropDuplicates().show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



There is a considerable in crease in test score post parameter tuning. A f1 score of 0.83 is pretty good. It is not too high to suggest overfitting. Also, the prediction label includes both majority and minority classes.

#### Parameter tuning for Decision Tree classifier:

In [ ]:
# param grid for random forest classifier
dt_paramGrid = ParamGridBuilder()\
              .addGrid(dt_model.maxDepth,[5, 10, 15])\
              .addGrid(dt_model.maxBins,[32, 50, 100])\
              .build()

In [ ]:
%%time
score_train_dt, score_test_dt, predict_train_dt, predict_test_dt, bestModel_dt = model_tuning(train, test, dt_model, dt_paramGrid, 5)

CPU times: user 29.5 s, sys: 12 s, total: 41.5 s
Wall time: 2h 13min 20s


In [ ]:
print("f1 train score: {}".format(score_train_dt))
print("f1 test score: {}".format(score_test_dt))
print("Best parameter for max depth: {}".format(bestModel_dt._java_obj.getMaxDepth()))
print("Best parameter for number of bins: {}".format(bestModel_dt._java_obj.getMaxBins()))

f1 train score: 0.903062864395
f1 test score: 0.799080156163
Best parameter for max depth: 15
Best parameter for number of bins: 32


In [ ]:
# check whether prediction consists of both labels.
predict_test_dt.select("prediction").dropDuplicates().show()

The Decision Tree classifier also improved a bit from 0.74 to 0.79.

#### Parameter tuning for GBT classifier:

In [9]:
# param grid for random forest classifier
gbt_paramGrid = ParamGridBuilder()\
                .addGrid(gbt_model.maxDepth,[10, 15])\
                .addGrid(gbt_model.maxBins,[50, 100])\
                .build()
            
                #.addGrid(gbt_model.maxIter,[20, 50, 100])\

In [ ]:
%%time
score_train_gbt, score_test_gbt, predict_train_gbt, predict_test_gbt, bestModel_gbt = model_tuning(train, test, gbt_model, gbt_paramGrid, 5)

CPU times: user 2.83 s, sys: 559 ms, total: 3.39 s
Wall time: 36min 36s


In [14]:
print("f1 score on train dataset: {}".format(score_train_gbt))
print("f1 score on test dataset: {}".format(score_test_gbt))
print("Best parameter for max depth: {}".format(bestModel_gbt._java_obj.getMaxDepth()))
print("Best parameter for number of bins: {}".format(bestModel_gbt._java_obj.getMaxBins()))
#print("Best parameter for max iterations: {}".format(bestModel_dt._java_obj.getMaxIter()))

f1 score on train dataset: 0.936469868395
f1 score on test dataset: 0.824570722945
Best parameter for max depth: 10
Best parameter for number of bins: 50


In [16]:
# check whether prediction consists of both labels.
predict_test_gbt.select("prediction").dropDuplicates().show()

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



The GBT classifier also improved post tuning from 0.77 to 0.82.

Overall, the Random Forest Classifier provides the best score at 0.8364.